In [1]:
import boto3
from boto3 import client
import json

In [2]:
s3 = boto3.resource('s3')

In [3]:
bucket_name = 'canopy-production-ml'

pc_bucket = s3.Bucket(bucket_name)

In [4]:
print(pc_bucket)

s3.Bucket(name='canopy-production-ml')


In [5]:
chips = []
for obj in pc_bucket.objects.all():
    if 'yes' in obj.key:
        chips.append(obj)

In [6]:
for chip in chips[0:4]:
    id = chip.key.split("/")[4]
    

In [7]:
j_file = "train_test_polygons.json"

with open(j_file, 'r') as j:
    train_test = json.loads(j.read())


In [10]:
len(train_test["train"])

36

In [11]:
len(train_test["test"])

61

In [12]:
chips[0]

s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3000.tif')

In [13]:
type(chips[0].key)

str

In [17]:
chips[0].key

'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3000.tif'

In [18]:
chips[0].key.split('/')[-1]

'101_1000_3000.tif'

In [15]:
dir(pc_bucket)

['Acl',
 'Cors',
 'Lifecycle',
 'LifecycleConfiguration',
 'Logging',
 'Notification',
 'Object',
 'Policy',
 'RequestPayment',
 'Tagging',
 'Versioning',
 'Website',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_name',
 'copy',
 'create',
 'creation_date',
 'delete',
 'delete_objects',
 'download_file',
 'download_fileobj',
 'get_available_subresources',
 'load',
 'meta',
 'multipart_uploads',
 'name',
 'object_versions',
 'objects',
 'put_object',
 'upload_file',
 'upload_fileobj',
 'wait_until_exists',
 'wait_until_not_exists']

In [16]:
help(pc_bucket.copy)

Help on method bucket_copy in module boto3.s3.inject:

bucket_copy(CopySource, Key, ExtraArgs=None, Callback=None, SourceClient=None, Config=None) method of boto3.resources.factory.s3.Bucket instance
    Copy an object from one S3 location to an object in this bucket.
    
    This is a managed transfer which will perform a multipart copy in
    multiple threads if necessary.
    
    Usage::
    
        import boto3
        s3 = boto3.resource('s3')
        copy_source = {
            'Bucket': 'mybucket',
            'Key': 'mykey'
        }
        bucket = s3.Bucket('otherbucket')
        bucket.copy(copy_source, 'otherkey')
    
    :type CopySource: dict
    :param CopySource: The name of the source bucket, key name of the
        source object, and optional version ID of the source object. The
        dictionary format is:
        ``{'Bucket': 'bucket', 'Key': 'key', 'VersionId': 'id'}``. Note
        that the ``VersionId`` key is optional and may be omitted.
    
    :type Key

In [34]:
def train_test_s3_copy(chips, j_file, bucket_name='canopy-production-ml',
                       base_path='chips/cloudfree-merge-polygons/split/'):
    
    bucket = s3.Bucket(bucket_name)
    
    with open(j_file, 'r') as j:
        
        train_test_file = json.loads(j.read())
        
#     for split in list(train_test.keys()):
        
#         for polygon in train_test[split]:

    length = len(chips)
            
    for i, chip in enumerate(chips, 1):
        print(f'Processing chip {i} of {length}', end='\r', flush=True)

        CopySource = {
            'Bucket': bucket_name,
            'Key': chip.key
        }

        polygon_id = int(chip.key.split("/")[5])

        filename = chip.key.split('/')[-1]

        if polygon_id in train_test_file["test"]:
            train_test = 'test'

            #s3.Object(bucket, base_path + 'test/').copy_from(CopySource=bucket + chip.key)

        else:
            train_test = 'train_val'

            #s3.Object(bucket, base_path + 'train_val/').copy_from(CopySource=bucket + chip.key)

        new_key = f'{base_path}{train_test}/{polygon_id}/{filename}'

        bucket.copy(CopySource, new_key)

In [36]:
j_file = "train_test_polygons.json"
train_test_s3_copy(chips, j_file)

In [22]:
print('s3://canopy-production-ml/' + 'chips/cloudfree-merge-polygons/split/')

s3://canopy-production-ml/chips/cloudfree-merge-polygons/split/
